# Test Model

In [8]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import jiwer
import numpy as np
import gc 
import librosa
import os 

# model_dir = "/mnt/5fc7fd01-6487-4f39-8109-556023ff1f7f/puc/7 sem/topicos/multipa/wav2vec2-large-xlsr-japlmthufiel-ipajaplmthufielta-nq-ns/checkpoint-100"
model_dir="/mnt/5fc7fd01-6487-4f39-8109-556023ff1f7f/puc/7 sem/topicos/multipa/wav2vec2-large-xlsr-japlmthufiel-ipajaplmthufielta-nq-ns"
# model_dir = "/mnt/5fc7fd01-6487-4f39-8109-556023ff1f7f/puc/7 sem/topicos/multipa/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns"

#pegar vocab.json da raiz e copia-lo pro wav2vec2-large-xlsr-japlmthufiel-ipajaplmthufielta-nq-ns

processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
cache_dir = './cache'

model.eval()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [ ]:
import os
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# Carregar o processador e o modelo
model_dir = "/mnt/5fc7fd01-6487-4f39-8109-556023ff1f7f/puc/7 sem/topicos/multipa/wav2vec2-large-xlsr-japlmthufiel-ipajaplmthufielta-nq-ns"
os.getenv('CAC')


processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
model.eval()

# Função para carregar e processar o áudio
def load_and_process_audio(audio_path, target_sampling_rate=16000):
    audio, sr = librosa.load(audio_path, sr=None)  # Carregar o áudio
    print(f"Sampling rate do áudio: {sr}")  # Verificar a taxa de amostragem
    if sr != target_sampling_rate:
        print(f"Reamostrando o áudio para {target_sampling_rate}Hz")  # Exibir reamostragem
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sampling_rate)  # Reamostrar se necessário
    return audio

# Função para transcrever o áudio para IPA
def transcribe_audio_to_ipa(audio_path):
    # Carregar e processar o áudio
    audio = load_and_process_audio(audio_path)

    # Verificar se o áudio tem o tamanho adequado para o modelo
    print(f"Tamanho do áudio (número de amostras): {len(audio)}")

    # Processar o áudio para o modelo
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    print(input_values)

    # Verificar a forma do tensor de entrada
    print(f"Forma do tensor de entrada: {input_values.shape}")

    # Fazer a predição
    with torch.no_grad():
        logits = model(input_values).logits
    print(logits)

    # Obter os índices das previsões
    predicted_ids = torch.argmax(logits, dim=-1)
    print(predicted_ids)

    # Decodificar para IPA (usando o processor.decode diretamente)
    transcription = processor.decode(predicted_ids[0])  # Usando o método decode diretamente no primeiro item

    return transcription

# Caminho da pasta contendo os arquivos de áudio
audio_folder = '/mnt/5fc7fd01-6487-4f39-8109-556023ff1f7f/puc/7 sem/topicos/multipa/cache/downloads/extracted/'

# Iterar sobre todos os arquivos de áudio na pasta
for filename in os.listdir(audio_folder):
    if filename.endswith(".mp3"):  # Verificar se é um arquivo mp3
        audio_path = os.path.join(audio_folder, filename)
        print(f"Processando arquivo: {filename}")
        
        # Transcrever o áudio
        ipa_transcription = transcribe_audio_to_ipa(audio_path)

        # Exibir a transcrição IPA
        print(f"Transcrição IPA para {filename}: {ipa_transcription}\n")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def transcribe_audio_to_ipa(audio_path):
    # Carregar e processar o áudio
    audio = load_and_process_audio(audio_path)

    # Verificar se o áudio tem o tamanho adequado para o modelo
    print(f"Tamanho do áudio (número de amostras): {len(audio)}")

    # Processar o áudio para o modelo
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    print(input_values)

    # Verificar a forma do tensor de entrada
    print(f"Forma do tensor de entrada: {input_values.shape}")

    # Fazer a predição
    with torch.no_grad():
        logits = model(input_values).logits
    print("logits: ",logits)

    # Obter os índices das previsões
    predicted_ids = torch.argmax(logits, dim=-1)
    print("predicted_ids: ",predicted_ids)

    # Decodificar para IPA (usando o processor.decode diretamente)
    transcription = processor.decode(predicted_ids[0])  # Usando o método decode diretamente no primeiro item

    return transcription

# Caminho da pasta contendo os arquivos de áudio
audio_folder = '/mnt/5fc7fd01-6487-4f39-8109-556023ff1f7f/puc/7 sem/topicos/multipa/cache/downloads/extracted/def0c51c4e5eae516227b49db5a07980edbfac43a9b8402e9f3b33954f29661d/ja_test_0/'

# Iterar sobre todos os arquivos de áudio na pasta
for filename in os.listdir(audio_folder):
    if filename.endswith(".mp3"):  # Verificar se é um arquivo mp3
        audio_path = os.path.join(audio_folder, filename)
        print(f"Processando arquivo: {filename}")
        
        ipa_transcription = transcribe_audio_to_ipa(audio_path)

        print(f"Transcrição IPA para {filename}: {ipa_transcription}\n")

        if(ipa_transcription != "[IPA]"):
            print(ipa_transcription)

Processando arquivo: common_voice_ja_32421686.mp3
Sampling rate do áudio: 32000
Reamostrando o áudio para 16000Hz
Tamanho do áudio (número de amostras): 112320
tensor([[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002]])
Forma do tensor de entrada: torch.Size([1, 112320])
logits:  tensor([[[-0.7411, -0.5865, -0.5853,  ..., -0.9725, -0.9877,  5.1098],
         [-0.7519, -0.6079, -0.5937,  ..., -0.9558, -0.9867,  5.1225],
         [-0.7545, -0.6072, -0.5875,  ..., -0.9328, -1.0163,  5.1364],
         ...,
         [-0.7533, -0.5968, -0.5863,  ..., -0.9473, -1.0130,  5.1442],
         [-0.7477, -0.6008, -0.5937,  ..., -0.9481, -1.0173,  5.1409],
         [-0.7471, -0.5950, -0.5829,  ..., -0.9518, -1.0144,  5.1376]]])
predicted_ids:  tensor([[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300,
         300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300,
         300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300,
         300, 300, 

KeyboardInterrupt: 

In [ ]:
# Exibir os resultados
languages = ["ja", "pl", "mt", "hu", "fi", "el"]
num_samples = 5

results = test_model_on_samples(model_dir, languages, num_samples, cache_dir)

for lang, metrics in results.items():
    print(f"\nResults for {lang}:")
    print(f"PER: {metrics['per']:.4f}")
    print(f"PFER: {metrics['pfer']:.4f}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



Testing language: ja
Dataset ja carregado do cache.
Sample 1: [PAD]
Sample 2: [PAD]
